# Build 3 region model - AB
https://github.com/Phyrise/nnUNet_translation/blob/master/notebooks/nnUNetv2_translation_tutorial_dualmod.ipynb

In [1]:
import os, glob, shutil, json
from pathlib import Path
import SimpleITK as sitk
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


## config

In [2]:
REGION = 'TH'
config = {
    "dataset_id": 298,  # Updated to 200 for CT noNorm
    "dataset_data_name": f"synthrad2025_task1_MR_{REGION}_zscore_synthrad_masked",
    "dataset_target_name": f"synthrad2025_task1_CT_{REGION}_zscore_synthrad_masked",
    "data_root": f"/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/{REGION}", # include centreD
    "preprocessing_CT": "CT_zscore_synthrad", 
    "preprocessing_MRI": "MR",
    "preprocessing_mask": "masked",
}

In [3]:
# save config to a JSON file
config_path = Path(f"config_{config['dataset_id']}.json")
with open(config_path, 'w') as f:
    json.dump(config, f, indent=4)

In [4]:

data_root = config["data_root"]
os.environ["nnUNet_raw"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw"
os.environ["nnUNet_preprocessed"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed"
os.environ["nnUNet_results"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results"

# example with 2 input modalities
list_data_mri = sorted(glob.glob(os.path.join(data_root, '**','mr.mha'), recursive=True))
list_data_mask = sorted(glob.glob(os.path.join(data_root, '**','mask.mha'), recursive=True))
list_data_ct = sorted(glob.glob(os.path.join(data_root, '**','ct.mha'), recursive=True))


print("input1 ---", len(list_data_mri), list_data_mri)
print("input2 ---", len(list_data_mask), list_data_mask)
print("target ---", len(list_data_ct), list_data_ct)

input1 --- 182 ['/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH/1THA001/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH/1THA002/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH/1THA003/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH/1THA004/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH/1THA005/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH/1THA010/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH/1THA011/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH/1THA013/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/TH/1T

## Define dataset ID and make paths

In [5]:
dataset_id = config["dataset_id"]
dataset_data_name = config["dataset_data_name"]
dataset_target_name = config["dataset_target_name"]

# we will copy the datas
# do not use exist_ok=True, we want an error if the dataset exist already
dataset_data_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id:03d}_{dataset_data_name}') 
os.makedirs(dataset_data_path, exist_ok = True)
os.makedirs(os.path.join(dataset_data_path, 'imagesTr'), exist_ok=True)
os.makedirs(os.path.join(dataset_data_path, 'labelsTr'), exist_ok = True)

dataset_target_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id+1:03d}_{dataset_target_name}') 
os.makedirs(dataset_target_path, exist_ok = True)
os.makedirs(os.path.join(dataset_target_path, 'imagesTr'), exist_ok = True)
os.makedirs(os.path.join(dataset_target_path, 'labelsTr'), exist_ok = True)

## Copy files and create dummy masks

In [6]:


def process_file_masked(data_path, dataset_path, modality_suffix="_0000", outsideValue=0):
    curr_img = sitk.ReadImage(data_path)
    mask_img = sitk.ReadImage(data_path.replace('mr.mha', 'mask.mha'))
    # values in the mask different from maskingValue are copied over, all other values are set to outsideValue
    masked_image = sitk.Mask(curr_img, mask_img, maskingValue=0, outsideValue=outsideValue)  
    filename = data_path.split(os.sep)[-2]
    if not filename.endswith(f'{modality_suffix}.mha'):
        filename = filename + f'{modality_suffix}.mha'
    sitk.WriteImage(masked_image, os.path.join(dataset_path, f'imagesTr/{filename}'))

    data = sitk.GetArrayFromImage(curr_img)
    data = np.ones_like(data)

    filename = filename.replace(modality_suffix, '')  # Remove modality suffix for masks
    label_path = os.path.join(dataset_path, f'labelsTr/{filename}')
    if not os.path.exists(label_path):
        label_img = sitk.GetImageFromArray(data)
        label_img.SetDirection(curr_img.GetDirection())
        label_img.SetOrigin(curr_img.GetOrigin())
        label_img.SetSpacing(curr_img.GetSpacing())
        sitk.WriteImage(label_img, label_path)

# Use the affine from the last MRI as a placeholder, but for sitk we use spacing/origin/direction from the image itself

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_file_masked(data_path, dataset_data_path, "_0000", 0), list_data_mri), total=len(list_data_mri)))

# with ThreadPoolExecutor() as executor:
#     list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0001"), list_data_mask), total=len(list_data_mask)))

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda target_path: process_file_masked(target_path, dataset_target_path, "_0000", -1000), list_data_ct), total=len(list_data_ct)))


  0%|          | 0/182 [00:00<?, ?it/s]

100%|██████████| 182/182 [01:12<00:00,  2.51it/s]askImageFilter (0x1555309f8ed0): Support for pixel type 16-bit signed integer for the MaskImage input has been deprecated and will be removed in future versions. Implicitly casting input to support 'sitkUInt8' type. MaskImageFilter (0x155519acced0): Support for pixel type 16-bit signed integer for the MaskImage input has been deprecated and will be removed in future versions. Implicitly casting input to support 'sitkUInt8' type. Filter (0x1555198cbed0): Support for pixel type 16-bit signed integer for the MaskImage input has been deprecated and will be removed in future versions. Implicitly casting input to support 'sitkUInt8' type. MaskImageFilter (0x155518ec6ed0): Support for pixel type 16-bit signed integer for the MaskImage input has been deprecated and will be removed in future versions. Implicitly casting input to support 'sitkUInt8' type. MaskImageFilter (0x15551b2d8ed0): Support for pixel type 16-bit signed integer for the MaskIm

## Create dataset.json

In [7]:
# /!\ you will need to edit this with regards to the number of modalities used;
data_dataset_json = {
    "labels": {
        "label_001": "1", 
        "background": 0
    },
    "channel_names": {
        "0": config["preprocessing_MRI"],
        # "1": config["preprocessing_mask"],
        
    },
    "numTraining": len(list_data_mri),
    "file_ending": ".mha"
}
dump_data_datasets_path = os.path.join(dataset_data_path, 'dataset.json')
with open(dump_data_datasets_path, 'w') as f:
    json.dump(data_dataset_json, f)

target_dataset_json = {
    "labels": {
        "label_001": "1",
        "background": 0
    },
    "channel_names": {
        "0": config["preprocessing_CT"],
    },
    "numTraining": len(list_data_ct),
    "file_ending": ".mha"
}
dump_target_datasets_path = os.path.join(dataset_target_path, 'dataset.json')
with open(dump_target_datasets_path, 'w') as f:
    json.dump(target_dataset_json, f)

## Apply preprocessing and unpacking

In [8]:
if 'MPLBACKEND' in os.environ: 
    del os.environ['MPLBACKEND'] # avoid conflicts with matplotlib backend  
    
os.system(f'nnUNetv2_plan_and_preprocess -d {dataset_id} -c 3d_fullres')
os.system(f'nnUNetv2_unpack {dataset_id} 3d_fullres 0')

os.system(f'nnUNetv2_plan_and_preprocess -d {dataset_id + 1} -c 3d_fullres')
os.system(f'nnUNetv2_unpack {dataset_id + 1} 3d_fullres 0')

Fingerprint extraction...
Dataset298_synthrad2025_task1_MR_TH_zscore_synthrad_masked
Experiment planning...
Using ZScoreNormalization for image normalization
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.   1.03 1.03]. 
Current patch size: (48, 192, 224). 
Current median shape: [107.5        462.13592233 520.38834951]
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.     1.0609 1.0609]. 
Current patch size: (48, 192, 224). 
Current median shape: [107.5        448.67565275 505.2314073 ]
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.       1.092727 1.092727]. 
Current patch size: (48, 192, 224). 
Current median shape: [107.5        435.60742985 490.51592941]
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.         1.12550881 1.12550881]. 
Current pa

100%|████████████████████████████████████████| 182/182 [04:46<00:00,  1.57s/it]


Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

Fingerprint extraction...
Dataset299_synthrad2025_task1_CT_TH_zscore_synthrad_masked
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer


100%|████████████████████████████████████████| 182/182 [01:05<00:00,  2.77it/s]


Experiment planning...
Using CTNormalizationClippingSynthrad2025 for image normalization
Using CTNormalizationClippingSynthrad2025 for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.   1.03 1.03]. 
Current patch size: (48, 192, 224). 
Current median shape: [107.5        462.13592233 520.38834951]
Using CTNormalizationClippingSynthrad2025 for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.     1.0609 1.0609]. 
Current patch size: (48, 192, 224). 
Current median shape: [107.5        448.67565275 505.2314073 ]
Using CTNormalizationClippingSynthrad2025 for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.       1.092727 1.092727]. 
Current patch size: (48, 192, 224). 
Current median shape: [107.5        435.60742985 490.51592941]
Using CTNormalizationClippingSynthrad2025 for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.         1.12550881 1.12550881]. 
Current patch s

100%|████████████████████████████████████████| 182/182 [06:33<00:00,  2.16s/it]


Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



0

## Copy mask raw data to the preprocessed folder


In [9]:
import shutil
def process_mask_file(data_path, dataset_mask_path):

    filename = data_path.split(os.sep)[-2]
    if not filename.endswith(f'.mha'):
        filename = filename + f'.mha'
    shutil.copy(data_path, os.path.join(dataset_mask_path, filename))


# Use the affine from the last MRI as a placeholder, but for sitk we use spacing/origin/direction from the image itself
dataset_mask_path = os.path.join(os.environ['nnUNet_preprocessed'], f'Dataset{dataset_id:03d}_{dataset_data_name}', 'masks')
os.makedirs(dataset_mask_path, exist_ok=True) 
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_mask_file(data_path, dataset_mask_path), list_data_mask), total=len(list_data_mask)))


100%|██████████| 182/182 [00:00<00:00, 1060.26it/s]


## Define 2nd modality raw data as gt_seg of 1st modality

In [10]:
nnunet_datas_preprocessed_dir = os.path.join(os.environ['nnUNet_preprocessed'], f'Dataset{dataset_id+1:03d}_{dataset_target_name}') 
nnunet_targets_preprocessed_dir = os.path.join(os.environ['nnUNet_preprocessed'], f'Dataset{dataset_id:03d}_{dataset_data_name}') 

list_targets = glob.glob(os.path.join(f"{dataset_target_path}/imagesTr", '*'))
list_targets.sort()
list_gt_segmentations_datas = glob.glob(os.path.join(f"{nnunet_targets_preprocessed_dir}/gt_segmentations", '*'))
list_gt_segmentations_datas.sort()

print(nnunet_targets_preprocessed_dir)

for (preprocessed_path, gt_path) in zip(list_targets, list_gt_segmentations_datas):
    # here, gt_path is the path to the gt_segmentation in nnUNet_preprocessed.
    print(preprocessed_path, "->", gt_path) # ensure correct file pairing; 
    shutil.copy(src = preprocessed_path, dst = gt_path) # we use shutil.copy to ensure safety, but switching to shutil.move would be more efficient

/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset298_synthrad2025_task1_MR_TH_zscore_synthrad_masked
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset299_synthrad2025_task1_CT_TH_zscore_synthrad_masked/imagesTr/1THA001_0000.mha -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset298_synthrad2025_task1_MR_TH_zscore_synthrad_masked/gt_segmentations/1THA001.mha
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset299_synthrad2025_task1_CT_TH_zscore_synthrad_masked/imagesTr/1THA002_0000.mha -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset298_synthrad2025_task1_MR_TH_zscore_synthrad_masked/gt_segmentations/1THA002.mha
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset299_synthrad2025_task1_CT_TH_zscore_synthrad_

## Define 2nd modality preprocessed files as ground truth of 1st modality


In [11]:
list_preprocessed_datas_seg_path = sorted(glob.glob(os.path.join(nnunet_targets_preprocessed_dir, 'nnUNetPlans_3d_fullres/*_seg.npy')))

list_preprocessed_targets_path = sorted(glob.glob(os.path.join(nnunet_datas_preprocessed_dir, 'nnUNetPlans_3d_fullres/*.npy')))
list_preprocessed_targets_path = [name for name in list_preprocessed_targets_path if '_seg' not in name]

for (datas_path, targets_path) in zip(list_preprocessed_datas_seg_path, list_preprocessed_targets_path):
    print(targets_path, "->", datas_path)
    shutil.copy(src = targets_path, dst = datas_path) 

/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset299_synthrad2025_task1_CT_TH_zscore_synthrad_masked/nnUNetPlans_3d_fullres/1THA001.npy -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset298_synthrad2025_task1_MR_TH_zscore_synthrad_masked/nnUNetPlans_3d_fullres/1THA001_seg.npy
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset299_synthrad2025_task1_CT_TH_zscore_synthrad_masked/nnUNetPlans_3d_fullres/1THA002.npy -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset298_synthrad2025_task1_MR_TH_zscore_synthrad_masked/nnUNetPlans_3d_fullres/1THA002_seg.npy
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset299_synthrad2025_task1_CT_TH_zscore_synthrad_masked/nnUNetPlans_3d_fullres/1THA003.npy -> /datasets/work/hb-synthrad2023/work/synthra

You should be able to start training with :
```
export nnUNet_raw="/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw"
export nnUNet_preprocessed="/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed"
export nnUNet_results="/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results"
```
nnUNetv2_train 200 3d_fullres 0 -tr nnUNetTrainerMRCT


In [ ]:
os.system(f'nnUNetv2_train {dataset_id} 3d_fullres 0 -tr nnUNetTrainerMRCT')

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

Setting signal to automatically requeue the job before timeout.
Start listening: waiting for signal

This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 2, 'patch_size': [48, 192, 224], 'median_image_size_in_voxels': [107.5, 476.0, 536.0], 'spacing': [3.0, 1.0, 1.0], 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [False], 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resample_data_or_s

Traceback (most recent call last):
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/envs/nnunet_trans2/bin/nnUNetv2_train", line 8, in <module>
    sys.exit(run_training_entry())
             ^^^^^^^^^^^^^^^^^^^^
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/ref/nnUNet_translation/nnunetv2/run/run_training.py", line 302, in run_training_entry
    run_training(args.dataset_name_or_id, args.configuration, args.fold, args.tr, args.p, args.pretrained_weights,
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/ref/nnUNet_translation/nnunetv2/run/run_training.py", line 238, in run_training
    nnunet_trainer.run_training()
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/ref/nnUNet_translation/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py", line 1300, in run_training
    train_outputs.append(self.train_step(next(self.dataloader_train)))
                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^


Caught signal: 15
Signal sigterm. 
Caught signal: 15
Signal sigterm. 
Caught signal: 15
Signal sigterm. 
Caught signal: 15
Signal sigterm. 
Caught signal: 15
Signal sigterm. 
Caught signal: 15
Signal sigterm. 
Caught signal: 15
Signal sigterm. 
Caught signal: 15
Signal sigterm. 
Caught signal: 15
Signal sigterm. 


: 